# Nashville Police Service Calls Analysis

## Dependencies

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

### Import

* N.B. - The dataset is large (more than 6.5M records), so it is not available in this Github repo.

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; If you'd like the dataset, you may find it [here](https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Calls-for-Servic/kwnd-qrrm), on the nashville.gov website.

In [ ]:
main_df = pd.read_csv('data/Metro_Nashville_Police_Department_Calls_for_Service.csv',
                      parse_dates=['Call Received'],
                     dtype={'Event Number': str,
                            'Complaint Number': float,
                            'Tencode': str,
                            'Tencode Description': str,
                            'Tencode Suffix': str,
                            'Tencode Suffix Description': str,
                            'Disposition Code': str,
                            'Disposition Description': str,
                            'Block': float,
                            'Street Name': str,
                            'Unit Dispatched': str,
                            'Shift': str,
                            'Sector': str,
                            'Zone': str,
                            'RPA': float,
                            'Latitude': float,
                            'Longitude': float,
                            'Mapped Location': str
                     })

### Preprocessing

* Let's get a sample of the data to see what we're working with.

In [ ]:
samp_df = main_df.sample(frac=.01, random_state=22)

In [ ]:
cols = ['Event Number', 'Call Received', 'Complaint Number', 'Tencode',
       'Tencode Description', 'Tencode Suffix', 'Tencode Suffix Description',
       'Disposition Code', 'Disposition Description', 'Block', 'Street Name',
       'Unit Dispatched', 'Shift', 'Sector', 'Zone', 'RPA', 'Latitude',
       'Longitude', 'Mapped Location']

In [ ]:
samp_df.dtypes

#### 'Event Number'

* It looks like all the event numbers begin with 'PD'; if this is the case, then I can strip those two characters and cast as an int, saving space

In [ ]:
pd_check = [event.startswith('PD') for event in samp_df['Event Number'].values]

In [ ]:
print(sum(pd_check))

In [ ]:
def event_number_clean(num):
    return int(num[2:])

In [ ]:
samp_df['Event Number'] = samp_df['Event Number'].apply(event_number_clean)

In [ ]:
samp_df.iloc[[0]]

#### 'Call Received'

* This is a datetime column, so I'll parse as I read in the csv

In [ ]:
samp_df['Call Received'].head(20)

#### 'Complaint Number'

* I am not interested in the specific number, just whether or not an incident was generated, so I'll update this to a simple Boolean flag

In [ ]:
samp_df['Complaint Number'].isna().value_counts()

In [ ]:
type(samp_df['Complaint Number'][5797724])

In [ ]:
def complaint_number_clean(num):
    if np.isnan(num):
        return 0
    else:
        return 1

In [ ]:
samp_df['generated_incident_yn'] = samp_df['Complaint Number'].apply(complaint_number_clean)
samp_df = samp_df.drop('Complaint Number', axis=1)

In [ ]:
# check the function
samp_df['generated_incident_yn'].value_counts()

In [ ]:
# Markdown shortcut!
for col in cols:
    print(f"#### '{col}'")

#### 'Tencode'

* This column seems to be clean. It's high-cardinality, and the codes are numeric, so I'll have to be careful with them if I do any modeling.

In [ ]:
samp_df['Tencode'].value_counts()

#### 'Tencode Description'

* It looks like there are some blanks here.
* However, the desciptions match the appendix in the metadata document, so rather than clogging the dataframe with strings, I'll remove this column.

In [ ]:
samp_df['Tencode Description'].isna().value_counts()

In [ ]:
samp_df[samp_df['Tencode Description'].notna()].head(20)

In [ ]:
samp_df = samp_df.drop('Tencode Description', axis=1)

#### 'Tencode Suffix'

* Like the tencode column, this is high-cardinality, though these are mostly strings instead of numeric values. For consistency, it looks like I should read the csv with the datatypes specified.

In [ ]:
type(samp_df['Tencode Suffix'][5797724])

In [ ]:
samp_df['Tencode Suffix'].value_counts()

#### 'Tencode Suffix Description'

* This one is like the tencode description, a string which is unneeded. I'll drop the column.

In [ ]:
samp_df = samp_df.drop('Tencode Suffix Description', axis=1)

#### 'Disposition Code'

* Interestingly, it would appear that tencode suffixes are sometimes appended to the disposition code instead of the tencode. That'll be fun to clean up!

* Here's my thinking:    
    * If the disposition code includes a letter which is not a valid option, and it is a valid option as a tencode suffix, and it is not already present in the tencode suffix, I'll append the letter to the tencode suffix.
    * If the disposition code includes a letter which is not a valid option, and it is *not* a valid option as a tencode suffix, I'll remove the letter.
    * If the disposition code includes a letter which is a valid option, and it is *also* a valid option as a tencode suffix, I'll do the following:
        * For A:
            * disposition codes 1, 4, 5, 7, 8, 9, 10, 11, 13, 14, 15 seem unlikely to result in arrest, so the A will be considered as a tencode and appended if not already present.
            * disposition codes 2, 3, 6, 12 seem like they could plausibly result in arrest, so the A will be left as part of the disposition code.
        * For C, O, and P:
            * if the letter is *not* included in the tencode suffix, I'll append it, while also leaving it as part of the disposition code.
            * if the letter is already included in the tencode suffix, I'll do nothing, leaving it in both places.

In [ ]:
def disposition_code_clean(row):
    
    # These three values should all be strings
    disp_code_letter = str(row['Disposition Code'])[-1:]
    disp_code_number = str(row['Disposition Code'])[:-1]
    tencode_suffix = str(row['Tencode Suffix'])
    
    if disp_code_letter.isnumeric():
        pass
    elif disp_code_letter not in ['A', 'C', 'O', 'P']:
        if disp_code_letter not in tencode_suffix:
            row['Tencode Suffix'] = tencode_suffix + disp_code_letter
        else:
            pass        
    else:
        if disp_code_letter == 'A':
            if disp_code_number in ['1', '4', '5', '7', '8', '9', '10', '11', '13', '14', '15']:
                if disp_code_letter not in tencode_suffix:
                    row['Tencode Suffix'] += disp_code_letter
                    row['Disposition Code'] = disp_code_number
                else:
                    row['Disposition Code'] = disp_code_number
            else:
                pass
        elif disp_code_letter in ['C', 'O', 'P']:
            if disp_code_letter not in tencode_suffix:
                row['Tencode Suffix'] += disp_code_letter
            else:
                pass

In [ ]:
clean_dispo = ambig_df.apply(disposition_code_clean, axis=1)

In [ ]:
ambig_df.reset_index(inplace=True)

In [ ]:
ambig_df

In [ ]:
pd.set_option("display.max_rows", 100)

In [ ]:
samp_df['Disposition Code'].value_counts()

In [ ]:
pd.reset_option("display.max_rows")

In [ ]:
ambig_df = samp_df[samp_df['Disposition Code'].str.contains('A|C|O|P', case=False, regex=True, na=False)]

#### 'Disposition Description'

In [ ]:
samp_df[samp_df['Disposition Code'] == '3A']

#### 'Block'

#### 'Street Name'

#### 'Unit Dispatched'

#### 'Shift'

#### 'Sector'

#### 'Zone'

#### 'RPA'

#### 'Latitude'

#### 'Longitude'

#### 'Mapped Location'

## EDA

In [ ]:
main_df.columns

In [ ]:
main_df.dtypes

In [ ]:
for col in cols:
    print(f'Column name: {col}')
    print(main_df[col].head(10))
    print('\n*******\n')